In [ ]:
import pandas as pd
import numpy as np
import scipy.optimize as opt
from scipy.sparse import lil_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Importing and Cleaning Data

## Loading and Cleaning the Preferences

In [ ]:
prefs_form = pd.read_csv("final_prefs2.csv") # from Google Forms
# All the columns in the preferences table have the workd "preferences" in them
lab_cols = [c for c in prefs_form.columns if "preferences for lab" in c.lower()]
disc_cols = [c for c in prefs_form.columns if "discussion" in c.lower()]
util_map = {
    "Cannot make this time.": 0.,
    "Least Preferred": 1.,
    "Moderately Preferred": 2.,
    "Strongly Preferred": 3.,
    "Strong Preferred": 3. #oops typo in form.
}
# Convert strings to utility scores
prefs_form[lab_cols] = prefs_form[lab_cols].replace(util_map)
prefs_form[disc_cols] = prefs_form[disc_cols].replace(util_map)
prefs_form.shape
prefs_form = prefs_form.set_index("Email Address")

## Loading the Roster

In [ ]:
roster = pd.read_csv("final_roster.csv") # from cal central or bcourses
roster = roster.set_index("Email Address")
roster.shape

In [ ]:
roster["Sections"].value_counts()

In [ ]:
# see how many enrolled and waitlisted students
roster["Role"].value_counts()

In [ ]:
# only get enrolled students
enrolled_roster = roster[roster["Role"] == "Student"]

In [ ]:
# left join roster with preferences to only get students who filled out form and are on the roster
enrolled_prefs = enrolled_roster.join(prefs_form, how="left")
enrolled_prefs[lab_cols + disc_cols] = enrolled_prefs[lab_cols + disc_cols].fillna(3)

In [ ]:
enrolled_prefs.shape

In [ ]:
lab_prefs = enrolled_prefs[lab_cols]
disc_prefs = enrolled_prefs[disc_cols]

In [ ]:
# Print dimensions
(n_students, n_labs) = lab_prefs.shape
(n_students, n_discussions) = disc_prefs.shape
print("Number of Students:", n_students)
print("Number of Labs Times:", n_labs)
print("Number of Discussion Slots:", n_discussions)

In [ ]:
# View the discussion time/location slots
disc_cols

In [ ]:
# View the lab time slots
lab_cols

# Setting The Section and Lab Sizes

This needs to be updated to reflect actual capacity or remaining capacity

In [ ]:
disc_room_size = 35
disc_ids = np.array([[101], [102, 129], [103], [104], [126, 127], [110], [122, 123], [121], [107], 
                     [108, 109], [111], [105, 112], [113], [131], [132], [114], [], [115, 135], 
                     [133], [134], [116], [117, 136, 138], [137], [118], [119, 139, 140], [120], 
                     [141], [130], [128], [124, 125]])
disc_capacity = np.array([disc_room_size * np.ones(len(d)) for d in disc_ids])
disc_slot_capacity = np.array([np.sum(s) for s in disc_capacity])


In [ ]:
lab_room_size = 35
lab_ids = np.array([[107, 108, 109, 110], [111, 112, 113, 114], [115, 116, 117, 118], [119, 120, 121, 122], 
                   [123, 124, 125, 126], [127, 128, 101, 102], [133], [134], [135, 136, 137, 138], 
                   [132, 129], [104, 103, 130], [106, 105, 131], []])
lab_capacity = np.array([lab_room_size * np.ones(len(d)) for d in lab_ids])
lab_slot_capacity = np.array([np.sum(s) for s in lab_capacity])

# Optimizing the Assignments

In [ ]:
def run_lp(prefs, cap, fuzzing=1.0e-5):
    """
    This function returns the optimal section assignments
    
    prefs is an n_students by n_sections matrix of utility values. Larger values are better.
    cap is an n_sections vector of section sizes.
    """
    
    (n,d) = prefs.shape
    
    # The optimizer MINIMIZES the sum of the weights.
    w = -prefs.flatten() # prefs is in row major form. 

    ### This is a bit of a hack but the problem is not well conditioned 
    ### so by adding a small amount of noise we ensure a single solution.
    
    w += fuzzing * np.random.randn(len(w)) 
    
    # The equality constraints enforces that every student is in one section
    # Each row in Aeq is the constrain for a single student
    
    Aeq = lil_matrix((n, n*d))
    for i in range(n):
        Aeq[i,(i*d):((i+1)*d)] = 1.
    Aeq = Aeq.asformat("csr")
    # Note we will also add a 0 < x < 1 constraint in the bounds arg to linprog
    
    # The inequality constraint ensures that no room has too many students
    Aub = lil_matrix((d, n*d))
    for i in range(d):
        tmp = np.zeros((n,d))
        tmp[:,i] = 1.
        Aub[i,:] = tmp.flatten()
    Aub = Aub.asformat("csr")
    
    options = dict(
        sparse=True, # Treat the constraint matrices as sparse
#         maxiter=100000,
#         tol=1.0e-10,
        disp=False)
    res = opt.linprog(w, Aub, cap, Aeq, np.ones(n), bounds = (0,1), options=options)
    return res

def compute_assignments(prefs, cap, unhappy = 0.0):
    """
    Compute the section assignments from the output of the optimization.
    prefs: pandas dataframes of section preferences used for the optimization
    cap: a numpy array of the section sizes
    """
    soln = run_lp(prefs.to_numpy(), cap)
    (n,d) = prefs.shape
    rounded_x = np.round(soln.x.reshape(n,d)) # Rounding the solution
    print("Over Capacity:", np.sum(rounded_x.sum(axis=0) > cap))
    print("Unassigned:", np.sum(rounded_x.sum(axis=1) != 1.))
    result = pd.DataFrame(
        {"Assignment": np.nonzero(rounded_x)[1], 
         "Happiness": prefs.to_numpy().flatten()[rounded_x.flatten() == 1.]}, 
        index=prefs.index)
    print("Unhappy Students:", np.sum(result['Happiness'] == 0))
    return result

In [ ]:
# Set random seed to get the same assignments
np.random.seed(4)

# Run LP to assign students to a time slot for lab
lab_opt = compute_assignments(lab_prefs, lab_slot_capacity)

In [ ]:
# Set random seed to get the same assignments
np.random.seed(4)

# Run LP to assign students to a time slot for lab
disc_opt = compute_assignments(disc_prefs, disc_slot_capacity)

# Analyze Assignments

List students which have a non-satisfiable assignment

In [ ]:
# View unhappy lab students
roster.loc[lab_opt[lab_opt["Happiness"] == 0].index, :]

In [ ]:
# View unhappy discussion students
roster.loc[disc_opt[disc_opt["Happiness"] == 0].index, :]

# Visualize Assignments

In [ ]:
# Plot distribution of happiness for lab
plt.figure(figsize=(10., 4.))
plt.subplot(1,2,1)
lab_opt["Happiness"].value_counts().sort_index().plot(kind="bar")
plt.title("Lab Happiness")
plt.ylabel("Number of Students")
plt.xlabel("Lab Happiness Score")
plt.subplot(1,2,2)
# Plot distribution of happiness for discussion
disc_opt["Happiness"].value_counts().sort_index().plot(kind="bar")
plt.title("Discussion Happiness")
plt.xlabel("Disc Happiness Score")
plt.savefig("enrolled_happiness.png")

# Assign Students to Physical Discussions

In [ ]:
def assign_rooms(opt, room_capacity, room_names):
    opt = opt.copy()
    room_capacity = np.array([c.copy() for c in room_capacity]) # deep copy
    opt["Room Name"] = 0
    for (label, row) in opt.iterrows():
        slot = int(row["Assignment"]);
        room_id = np.argmax(room_capacity[slot])
        room_capacity[slot][room_id] -= 1
        opt.loc[label, "Room Name"] = room_names[slot][room_id]
    return (opt, room_capacity)

In [ ]:
(disc_opt, disc_capacity_remaining) = assign_rooms(disc_opt, disc_capacity, disc_ids)
(lab_opt, lab_capacity_remaining) = assign_rooms(lab_opt, lab_capacity, lab_ids)

In [ ]:
roster.loc[disc_opt.index, "Disc. Room"] = disc_opt["Room Name"]
roster.loc[disc_opt.index, "Disc. Happiness"] = disc_opt["Happiness"]
roster.loc[lab_opt.index, "Lab Room"] = lab_opt["Room Name"]
roster.loc[lab_opt.index, "Lab Happiness"] = lab_opt["Happiness"]

# Matching Waitlisted Students

In [ ]:
# only get waitlisted students
waitlisted_roster = roster[roster["Role"] == "Waitlist Student"]

In [ ]:
# inner join roster with preferences to only get students who filled out form and are on the roster
waitlisted_prefs = waitlisted_roster.join(prefs_form, how="left")
waitlisted_prefs[lab_cols + disc_cols] = waitlisted_prefs[lab_cols + disc_cols].fillna(3)
waitlisted_lab_prefs = waitlisted_prefs[lab_cols]
waitlisted_disc_prefs = waitlisted_prefs[disc_cols]

In [ ]:
# Set random seed to get the same assignments
np.random.seed(4)
lab_room_padding = 3

lab_capacity_remaining_padded = np.array([
    c + lab_room_padding for c in lab_capacity_remaining])
lab_slot_capacity_reamining = np.array([np.sum(c) for c in lab_capacity_remaining_padded])

waitlisted_lab_opt = compute_assignments(waitlisted_lab_prefs, lab_slot_capacity_reamining)

In [ ]:
# Set random seed to get the same assignments
np.random.seed(4)
disc_room_padding = 3

disc_capacity_remaining_padded = np.array([
    c + disc_room_padding for c in disc_capacity_remaining])
disc_slot_capacity_reamining = np.array([np.sum(c) for c in disc_capacity_remaining_padded])

waitlisted_disc_opt = compute_assignments(waitlisted_disc_prefs, disc_slot_capacity_reamining)

In [ ]:
roster.loc[waitlisted_lab_opt[waitlisted_lab_opt["Happiness"] == 0].index, :]

In [ ]:
roster.loc[waitlisted_disc_opt[waitlisted_disc_opt["Happiness"] == 0].index, :]

In [ ]:
(waitlisted_disc_opt, disc_capacity_remaining_final) = assign_rooms(waitlisted_disc_opt, disc_capacity_remaining_padded, disc_ids)
(waitlisted_lab_opt, lab_capacity_remaining_final) = assign_rooms(waitlisted_lab_opt, lab_capacity_remaining_padded, lab_ids)

In [ ]:
roster.loc[waitlisted_disc_opt.index, "Disc. Room"] = waitlisted_disc_opt["Room Name"]
roster.loc[waitlisted_disc_opt.index, "Disc. Happiness"] = waitlisted_disc_opt["Happiness"]
roster.loc[waitlisted_lab_opt.index, "Lab Room"] = waitlisted_lab_opt["Room Name"]
roster.loc[waitlisted_lab_opt.index, "Lab Happiness"] = waitlisted_lab_opt["Happiness"]

In [ ]:
# Plot distribution of happiness for lab
plt.figure(figsize=(10., 4.))
plt.subplot(1,2,1)
roster["Lab Happiness"].value_counts().sort_index().plot(kind="bar")
plt.title("Lab Happiness")
plt.ylabel("Number of Students")
plt.xlabel("Lab Happiness Score")
plt.subplot(1,2,2)
# Plot distribution of happiness for discussion
roster["Disc. Happiness"].value_counts().sort_index().plot(kind="bar")
plt.title("Discussion Happiness")
plt.xlabel("Disc Happiness Score")
plt.savefig("overall_happiness.png")

In [ ]:
roster.columns

In [ ]:
columns_needed = ["Name", "Role", "Lab Room", "Disc. Room", "Lab Happiness", "Disc. Happiness"]
(
    roster[columns_needed].drop(columns = ["Disc. Room"])
        .sort_values(["Lab Room", "Role", "Name"])
        .to_csv("LabAssignments.csv")
)
(
    roster[columns_needed].drop(columns = ["Lab Room"])
        .sort_values(["Disc. Room", "Role", "Name"])
        .to_csv("DiscussionAssignments.csv")
)